In [1]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [6]:
# data/Original 存放原始数据
# data/Original/Training 训练集
# data/Original/Test 测试集

# data/Training_set 存放训练集，Training_set.csv为最终数据，前63列为21个三维骨骼坐标，64列为标记
# data/Test_set 存放训练集，Test_set.csv为最终数据，前63列为21个三维骨骼坐标，64列为标记

# label 0 for pencil down/ 1 for pencil up, 61 for training set/ 11 for test set

# txt->json->csv

txt_folder='data/Original/Test/Pencil_up/txt/'
json_folder='data/Original/Test/Pencil_up/json/'
csv_folder='data/Original/Test/Pencil_up/csv/'

idx=0

In [8]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=1) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # Flip the image horizontally for a later selfie-view display, and convert the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        
        if cv2.waitKey(100) & 0xFF == ord('b'): # 按下 b 键进行一次捕获
          with open(txt_folder+str(idx)+'.txt','w') as f:
            print(hand_landmarks,file=f)
            print(idx)
            idx+=1

    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(100) & 0xFF == 27: # 按下 esc 退出
      break
cap.release()

0
1
2
3
4
5
6
7
8
9
10


In [3]:
import json
import pandas as pd
from os import listdir
import re
import time
import numpy as np

def StandardScaler(x):
    '''数据标准化，且以21个骨骼点的第一个为原点'''
    x_array=np.array(x)
    x_array-=x_array[0]

    mean=np.mean(x_array)
    std=np.std(x_array)

    x_array-=mean
    x_array/=std

    x_corr=x_array.tolist()

    return x_corr



seq = re.compile(":")

t=time.time()

# txt->json
for file in listdir(txt_folder):
    result = []
    with open(txt_folder+file) as f:
        for line in f:
            lst = seq.split(line.strip())
            if (len(lst)>=2): 
                item = {str(lst[0]): float(lst[1])}
                result.append(item)    

    with open(json_folder+file[:-3]+'json', 'w') as dump_f:
        json.dump(result,dump_f)
    
# json->csv
for file in listdir(json_folder):
    with open(json_folder+file) as f:
        data = json.load(f)
        x = []
        y = []
        z = []
        xyz=[]
        for pt in data:
            if 'x' in pt: x.append(pt['x'])
            if 'y' in pt: y.append(pt['y'])
            if 'z' in pt: z.append(pt['z'])
        
        # 数据标准化
        x=StandardScaler(x)
        y=StandardScaler(y)
        z=StandardScaler(z)

        xyz=x+y+z # 将21*3=63个值排成一列，便于下一步训练

    df = pd.DataFrame(data=xyz)
    df.to_csv(csv_folder+file[:-4]+'csv')

print(time.time()-t)



0.11936092376708984


In [7]:
import numpy as np
import csv
from os import listdir
import pandas as pd

data=np.zeros((1,63)) # 存储最终结果

for file in listdir(csv_folder):
    cur_data=[]
    with open(csv_folder+file,'r',newline='') as csvfile:
        reader=csv.reader(csvfile)
        for row in reader:
            cur_data.append(row)

    arr_data=np.array(cur_data)
    arr_data=np.delete(arr_data,0,axis=0) # 删去索引
    arr_data=np.delete(arr_data,0,axis=1) # 删去索引

    arr_data=arr_data.transpose()

    data=np.row_stack((data,arr_data))

data=np.delete(data,0,axis=0) # 删去索引

labels=np.ones((11,1)) # label 0 for pencil down/ 1 for pencil up, 61 for training set/ 11 for test set

data=np.column_stack((data,labels))

df = pd.DataFrame(data=data)
df.to_csv('test_pencil_up.csv',index=False) # 暂存于主路径，后续需要将up与down合在一起



In [18]:
# test
a=np.zeros((3,3))
b=np.ones((1,3))


np.row_stack((a,b))
print(a)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
